### Identifying country names from incomplete house addresses

#### 1. Import Dependencies

In [1]:
import os
import zipfile
import pandas as pd 
from pathlib import Path 
from arcgis.gis import GIS
from arcgis.learn import prepare_textdata 
from arcgis.learn.text import TextClassifier

#### 2. Data Preparation

In [2]:
filepath = 'country_classifier.zip'

In [3]:
with zipfile.ZipFile(filepath, 'r') as zip_ref:
    zip_ref.extractall(Path(filepath).parent)

In [4]:
DATA_ROOT = Path(os.path.join(os.path.splitext(filepath)[0]))

In [5]:
data = prepare_textdata(DATA_ROOT, "classification", train_file="house-addresses.csv", 
                        text_columns="Address", label_columns="Country", batch_size=64)

C:\Users\aviparna.biswas\Anaconda3\envs\docuchief\lib\site-packages\arcgis\learn\models\_arcgis_model.py:450: UserWarning: Cuda is not available
  warnings.warn("Cuda is not available")


Exception: Traceback (most recent call last):

  File "C:\Users\aviparna.biswas\Anaconda3\envs\docuchief\lib\site-packages\arcgis\learn\_utils\text_data.py", line 32, in <module>
    from .text_transforms import (

  File "C:\Users\aviparna.biswas\Anaconda3\envs\docuchief\lib\site-packages\arcgis\learn\_utils\text_transforms.py", line 68, in <module>
    class TransformersBaseTokenizer(BaseTokenizer):

  File "C:\Users\aviparna.biswas\Anaconda3\envs\docuchief\lib\site-packages\arcgis\learn\_utils\text_transforms.py", line 77, in TransformersBaseTokenizer
    seq_len=512,

NameError: name 'PreTrainedTokenizer' is not defined


This module requires fastai, PyTorch and transformers as its dependencies.
Install them using - 'conda install -c esri -c fastai -c pytorch arcgis=1.8.2 scikit-image=0.15.0 pillow=6.2.2 libtiff=4.0.10 fastai=1.0.60 pytorch=1.4.0 torchvision=0.5.0 scikit-learn=0.23.1 --no-pin'
'conda install gdal=2.3.3'
'pip install transformers==3.3.0'

In [8]:
print(data.classes)

NameError: name 'data' is not defined

In [ ]:
data.show_batch(10) 

### 3. TextClassifier Model

In [9]:
print(TextClassifier.supported_backbones)

['BERT', 'RoBERTa', 'DistilBERT', 'ALBERT', 'FlauBERT', 'CamemBERT', 'XLNet', 'XLM', 'XLM-RoBERTa', 'Bart', 'ELECTRA', 'Longformer', 'MobileBERT', 'Funnel']


In [11]:
print(TextClassifier.available_backbone_models("xlm-roberta"))

Exception: Traceback (most recent call last):

  File "C:\Users\aviparna.biswas\Anaconda3\envs\docuchief\lib\site-packages\arcgis\learn\text\_text_classifier.py", line 23, in <module>
    from transformers import AutoTokenizer, AutoConfig

  File "C:\Users\aviparna.biswas\Anaconda3\envs\docuchief\lib\site-packages\transformers\__init__.py", line 124, in <module>
    from .pipelines import (

  File "C:\Users\aviparna.biswas\Anaconda3\envs\docuchief\lib\site-packages\transformers\pipelines.py", line 47, in <module>
    from .modeling_tf_auto import (

  File "C:\Users\aviparna.biswas\Anaconda3\envs\docuchief\lib\site-packages\transformers\modeling_tf_auto.py", line 45, in <module>
    from .modeling_tf_albert import (

  File "C:\Users\aviparna.biswas\Anaconda3\envs\docuchief\lib\site-packages\transformers\modeling_tf_albert.py", line 43, in <module>
    from .modeling_tf_utils import (

  File "C:\Users\aviparna.biswas\Anaconda3\envs\docuchief\lib\site-packages\transformers\modeling_tf

### 4. Load Model Architecture

In [ ]:
model = TextClassifier(data, backbone="xlm-roberta-base")

### 5. Model Training

In [ ]:
model.lr_find()

In [ ]:
model.fit(epochs=4, lr=0.001)

In [ ]:
# Unfreeze earlier layers to further fine-tune the model
model.unfreeze()

model.fit(epochs=6)

### 6. Validate the results

In [ ]:
# After training, checking the results to see how it performs
model.show_results(15)

### 7. Test the model prediction on an input text

In [ ]:
text = """1016, 8A, CL RICARDO LEON - SANTA ANA (CARTAGENA), 30319"""
print(model.predict(text))

### 8. Model metrics

In [ ]:
# Check to see how accurate the model is in correctly predicting the classes in the dataset
model.accuracy()

In [ ]:
# Also check precision, recall & f1 scores per label/class
model.metrics_per_label()

### 9. Get misclassified records

In [12]:
# To get the misclassified records we will call the model's get_misclassified_records method
misclassified_records = model.get_misclassified_records()

In [ ]:
misclassified_records.style.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])\
        .set_properties(**{'text-align': "left"}).hide_index()

### 10. Saving the trained model

In [ ]:
# Save the model for interfencing on unseen data
model.save("country_classifier")

### 11. Model Inference

In [ ]:
# Using the trained model to classify new text documents using the predict method
text_list = data._train_df.sample(15).Address.values
result = model.predict(text_list)

df = pd.DataFrame(result, columns=["Address", "CountryCode", "Confidence"])

df.style.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])\
        .set_properties(**{'text-align': "left"}).hide_index()